In [234]:
import numpy as np
import pandas as pd
from itertools import product
#from nltk import ngrams
#import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [320]:
#for 95% probability
_T = 2

# Calculate n grams

In [236]:
alphabet = 'абвгдеёжзийклмнопрстуфхцчьыъшщэюя'
n = 2

def build_n_grams(n):
    combinations = [''.join(val) for val in [p for p in product(list(alphabet),repeat = n)]]
    return combinations

n_grams = build_n_grams(n)

# Read textes and build dataframe

In [237]:
def build_dataframe(text: str, gramms):
    text_data = {}
    lenght = len(text)
    for i in gramms:
        text_data[i] = text.count(i)/lenght
    return text_data

In [238]:
def read_text(filepath, n_grams):
    data = {}
    with open(filepath, 'r') as f:
        try:
            text = ''.join(filter(str.isalpha, f.read()))
            data['lenght'] = len(text)
            data.update(build_dataframe(text, n_grams))
        except UnicodeDecodeError as e: 
            print('Unable to read the text (some encoding errors)')
            print('\033[91m {} \033[0m'.format(str(e)))
    return data

In [239]:
def read_authors(dirname, n_grams):
    data = []
    for _dir in os.listdir(dirname):
        _current_dir = os.path.join(dirname,_dir)
        if(os.path.isdir(_current_dir)):
            for _file in os.listdir(_current_dir):
                if '.DS_Store' in _file:
                    print('\033[91m {} \033[0m'.format(str('.DS_Store file')))
                else:
                    filepath = os.path.join(_current_dir, _file)
                    text = None
                    string_data = {'author': _dir, 'name': _file}
                    string_data.update(read_text(filepath, n_grams))
                    data.append(string_data)
    return data

# Read data for prediction 

In [240]:
def read_unknown(path, n_grams):
    data = []
    for _file in os.listdir(path):
        if '.DS_Store' in _file:
            print('\033[91m {} \033[0m'.format(str('.DS_Store file')))
        else:
            row = {'name': _file}
            row.update(read_text(os.path.join(path, _file), n_grams))
            data.append(row)
    return data

# Build model

In [244]:
data = read_authors('./20', n_grams)
df = pd.DataFrame(data)
unique_authors = df.author.unique()

 .DS_Store file 
 .DS_Store file 
 .DS_Store file 
 .DS_Store file 
 .DS_Store file 
 .DS_Store file 
 .DS_Store file 
 .DS_Store file 
 .DS_Store file 


In [245]:
unique_authors

array(['Хаксли', 'Ильин', 'Калинина', 'Курпатов', 'Агапкин', 'Марк Фрост',
       'Достоевский', 'Тургенев', 'Генри Миллер', 'Александрова',
       'Казанцев'], dtype=object)

In [246]:
df.head()

,author,lenght,name,аа,аб,ав,аг,ад,ае,аж,...,ёч,ёш,ёщ,ёъ,ёы,ёь,ёэ,ёю,ёя,ёё
0,Хаксли,16511,Хаксли гений и богиня часть 5,0.000242,0.001938,0.004542,0.001514,0.003392,0.002241,0.001575,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Хаксли,18174,Хаксли. Гений и богиня часть 1,0.000770,0.001871,0.003852,0.001211,0.002751,0.001706,0.001431,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Хаксли,17144,Хаксли гений и богиня часть 4,0.000875,0.001750,0.003266,0.000875,0.002158,0.001400,0.001400,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Хаксли,16904,Хаксли Слепец в Газе часть 1,0.000355,0.001420,0.005147,0.001538,0.002603,0.002544,0.001656,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Хаксли,16456,Хаксли о дивный новый мир часть 6,0.000122,0.001823,0.003889,0.001337,0.003281,0.001398,0.002613,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [333]:
def build_mean(df, authors):
    mean_authors = {author : [] for author in authors}
    df_mean = df.groupby('author').mean()
    for author in authors:
        idx = np.where(df_mean.index.values == author)[0][0]
        mean_authors[author] = df_mean.values[idx,1:]
    return mean_authors

In [334]:
df_mean = build_mean(df, unique_authors)
df_mean

{'Хаксли': array([0.00031533, 0.00177903, 0.00439904, ..., 0.        , 0.        ,
        0.        ]),
 'Ильин': array([0.00016143, 0.00178386, 0.00483303, ..., 0.        , 0.        ,
        0.        ]),
 'Калинина': array([0.00022998, 0.00204952, 0.00608202, ..., 0.        , 0.        ,
        0.        ]),
 'Курпатов': array([0.00014818, 0.00136453, 0.0038245 , ..., 0.        , 0.        ,
        0.        ]),
 'Агапкин': array([0.00024623, 0.00250982, 0.00301746, ..., 0.        , 0.        ,
        0.        ]),
 'Марк Фрост': array([0.00010484, 0.00158462, 0.00434727, ..., 0.        , 0.        ,
        0.        ]),
 'Достоевский': array([0.00015938, 0.00135644, 0.00479963, ..., 0.        , 0.        ,
        0.        ]),
 'Тургенев': array([0.00012003, 0.00114977, 0.00606656, ..., 0.        , 0.        ,
        0.        ]),
 'Генри Миллер': array([0.00013826, 0.00151943, 0.00438973, ..., 0.        , 0.        ,
        0.        ]),
 'Александрова': array([0.0002083

In [335]:
def build_confidence_interval(df, mean_values, authors):
    intervals = {author : [] for author in authors}
    for author in authors:
        author_mean = mean_values[author]
        author_intervals = []
        for _i, row in df.loc[df.author == author].iterrows():
            author_intervals.append(np.abs(author_mean-row[3:]))
            
        intervals[author] = _T*np.mean(author_intervals, axis=0)
    return intervals

In [337]:
confidence_interval = build_confidence_interval(df, df_mean, unique_authors)
confidence_interval

{'Хаксли': array([0.00042417666285501156, 0.0004978206857208515,
        0.0012464776526514437, ..., 0.0, 0.0, 0.0], dtype=object),
 'Ильин': array([0.00018210741781276177, 0.0005825240917857924,
        0.0007863891917465158, ..., 0.0, 0.0, 0.0], dtype=object),
 'Калинина': array([0.0002189171764858042, 0.0007971360189958723,
        0.0012595171761441832, ..., 0.0, 0.0, 0.0], dtype=object),
 'Курпатов': array([0.00011053184379196152, 0.0008384701316026313,
        0.001654193096001344, ..., 0.0, 0.0, 0.0], dtype=object),
 'Агапкин': array([0.00022160071790838643, 0.0009483377531415248,
        0.0012522183976239833, ..., 0.0, 0.0, 0.0], dtype=object),
 'Марк Фрост': array([0.00010961472028191854, 0.0005940366155850501,
        0.000894335101875249, ..., 0.0, 0.0, 0.0], dtype=object),
 'Достоевский': array([0.00020463277444388397, 0.0005330542781135433,
        0.0012647342085942257, ..., 0.0, 0.0, 0.0], dtype=object),
 'Тургенев': array([0.00013982152418814104, 0.0006527634613814976

# Prediction

In [338]:
predict = read_unknown('./predict20', n_grams)

 .DS_Store file 


In [340]:
df_predict = pd.DataFrame(predict)
df_predict.head()

,lenght,name,аа,аб,ав,аг,ад,ае,аж,аз,...,ёч,ёш,ёщ,ёъ,ёы,ёь,ёэ,ёю,ёя,ёё
0,16177,марк фрост. пробуждение часть 3,0.000124,0.000742,0.004389,0.001051,0.002967,0.000680,0.001113,0.003771,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,17510,марк фрост. список семи часть 1,0.000171,0.001371,0.006054,0.001142,0.002342,0.001542,0.001885,0.004512,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,14335,Самое главное о сердце и сосудах. С. Агапкин. ...,0.000767,0.003139,0.002721,0.002721,0.000907,0.001186,0.001116,0.002930,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,17506,Самое главное о сердце и сосудах. С. Агапкин. ...,0.000228,0.002399,0.003713,0.001714,0.001257,0.002513,0.001485,0.002456,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,16258,Тургенев. Отцы и дети. часть 1,0.000123,0.000923,0.004490,0.000923,0.005167,0.000861,0.001722,0.004244,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [373]:
def predict_author(data_text, intervals, means):
    authors_total = {key: 0 for key in means.keys()}
    for _author in means:
        diff = len(list(filter(lambda x: (x < 0), np.array(intervals[_author]) - np.abs(means[_author]-data_text))))
        authors_total[_author] = diff/len(data_text)
    #print(authors_total)
    return authors_total

In [377]:
for _i, row in df_predict.iterrows():
    _title = row.values[1]
    data_row = row.values[2:]
    predictions = predict_author(data_row, confidence_interval, df_mean)
    print(_title,' : ',  min(predictions, key = lambda x: predictions.get(x)))

марк фрост. пробуждение часть 3  :  Марк Фрост
марк фрост. список семи часть 1  :  Марк Фрост
Самое главное о сердце и сосудах. С. Агапкин. часть 3  :  Агапкин
Самое главное о сердце и сосудах. С. Агапкин. часть 2  :  Агапкин
Тургенев. Отцы и дети. часть 1  :  Тургенев
Казанцев К. Ненасытные часть 6  :  Казанцев
Курпатов А. Чертоги разума часть 2  :  Курпатов
Хаксли. гений и богиня часть3  :  Хаксли
Курпатов А. Чертоги разума часть 3  :  Курпатов
Курпатов А. Чертоги разума часть4  :  Курпатов
александрова. китайская кукла часть 1  :  Александрова
марк фрост. пробуждение часть 1  :  Марк Фрост
Ильин А. Маска резидента часть 7  :  Ильин
Хаксли. Гений и богиня часть 2  :  Достоевский
Ильин А. Маска резидента часть 8  :  Ильин
Казанцев К. Ненасытные часть 5  :  Казанцев
Тургенев. Муму  :  Достоевский
Александрова. Венец гекаты часть 1  :  Александрова
Курпатов А. Чертоги разума чать 1  :  Курпатов
